In [1]:
#! pip install climetlab_s2s_ai_competition --quiet

In [2]:
import climetlab as cml 

module 'Magics' has no attribute 'strict_mode'


In [3]:
import climetlab_s2s_ai_competition
print(f'Climetlab version : {cml.__version__}')
print(f'Climetlab-s2s-ai-competition plugin version : {climetlab_s2s_ai_competition.__version__}')

Climetlab version : 0.2.2
Climetlab-s2s-ai-competition plugin version : 0.2.2


# Using Zarr data

In [4]:
version = '0.1.20' # version of the data

In [5]:
import pandas as pd
date=pd.date_range(start="2020-01-02", end="2020-12-31", freq="w-thu")
#date = [d.strftime('%Y%m%d') for d in date]
date = [d.to_numpy() for d in date]

In [6]:
import numpy as np
date = [np.datetime64('2020-01-02'),'2020-01-09','20200116']
date

[numpy.datetime64('2020-01-02'), '2020-01-09', '20200116']

Let us get the zarr pointer to the data for realtime data. The "date" argument also accept lists of values.

In [ ]:
rt = cml.load_dataset("s2s-ai-competition-reference-set",
                         date=date,
                         version=version,
                         #hindcast=True,
                         format='zarr').to_xarray()

By downloading data from this dataset, you agree to the their terms: Attribution 4.0 International(CC BY 4.0). If you do not agree with such terms, do not download the data. For more information, please visit https://www.ecmwf.int/en/terms-use and https://apps.ecmwf.int/datasets/data/s2s/licence/.


Hindcast data are retrieved with hindcast=True.

In [ ]:
hc = cml.load_dataset("s2s-ai-competition-reference-set",
                         date=date,
                         version='0.1.20',
                         hindcast=True,
                         format='zarr').to_xarray()

This climetlab dataset can be used as a xarray.Dataset :

In [ ]:
rt

In [ ]:
hc

### Computing average and plotting

In [ ]:
#ds = rt.to_xarray()
cmlds = cml.load_dataset("s2s-ai-competition-reference-set",
                         date=date[0:2],
                         version='0.1.20',
                         hindcast=True,
                         format='zarr')
ds = cmlds.to_xarray()

In [ ]:
mean1 = ds.mean(dim="step")
cml.plot_map(mean1.isel(forecast_time=0, realization=0))

In [ ]:
mean2 = ds.mean(dim="forecast_time")
cml.plot_map(mean2.isel(step=20, realization=0))

In [ ]:
#mean3 = ds.mean(dim="time")
#cml.plot_map(mean3.isel(forecast_time=20, realization=0))

In [ ]:
mean3 = ds.mean(dim=["forecast_time", "step"])
cml.plot_map(mean3.isel(realization=0))

In [ ]:
mean3 = ds.mean(dim=["forecast_time", "step", "realization"])
cml.plot_map(mean3)

In [ ]:
cmlds = cml.load_dataset("s2s-ai-competition-reference-set",
                         date=date[0:1],
                         version='0.1.20',
                         hindcast=True,
                         format='zarr')

In [ ]:
ds = cmlds.to_xarray()
ds

In [ ]:
mean3 = ds.mean(dim=["step"])
#mean3 = ds.mean(dim=["step", "realization"])
cml.plot_map(mean3.isel(forecast_time=1))
#cml.plot_map(mean3.isel(realization=0,forecast_time=1))